In [1]:
import pandas as pd  
import pyedflib #library for efd files for eeg
from sklearn.model_selection import train_test_split 


from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from IPython.display import display
from sklearn import metrics
from pandas_summary import DataFrameSummary

import os
from os import scandir, getcwd
from os.path import abspath
#libreria para leer los ficheros
import pyedflib
import seaborn as sns
from tqdm import tqdm_notebook

%matplotlib inline
from matplotlib import pyplot as plt
plt.style.use(['seaborn-darkgrid'])
plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.family'] = 'DejaVu Sans'

from sklearn import metrics
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix

RANDOM_STATE = 17

In [2]:
#Importar datos, la fuente es https://www.isip.piconepress.com/projects/tuh_eeg/downloads/tuh_eeg_epilepsy/

In [3]:
def lsAb(ruta = getcwd()):
	return [abspath(arch.path) for arch in scandir(ruta) if arch.is_file()]

def ls(ruta = getcwd()):
	return [arch.name for arch in scandir(ruta) if arch.is_file()]

In [136]:

noEpi=lsAb("/noepi") #carpeta con archivos edf de no epilepticos para entrenar el modelo
MasterTable =[]
Etiqueta = []
contador_train=0
for file in noEpi:
	if (file!="/noepi/.DS_Store"):
#		print(file)
		contador_train+=1
		miEDFReader = pyedflib.EdfReader(file)
		Etiqueta.append("noepi")
		MasterTable.append(miEDFReader.readSignal(0,500,2500))
		miEDFReader.close

Epi=lsAb("/epi") #carpeta con archivos edf de epilepticos para entrenar el modelo
#ListaEdEpifReader =[]
for file in Epi:
	if (file!="/epi/.DS_Store"):
#		print(file)
		contador_train+=1
		miEDFReader = pyedflib.EdfReader(file)
		Etiqueta.append("epi")
		MasterTable.append(miEDFReader.readSignal(0,500,2500))
		miEDFReader.close
        
#ahora los datos de test
noEpi=lsAb("/test_noepis") #carpeta con archivos edf de no epilepticos para testar el modelo
contador_test=0
for file in noEpi:
	if (file!="/test_noepis/.DS_Store"):
#		print(file)
		contador_test+=1
		miEDFReader = pyedflib.EdfReader(file)
		Etiqueta.append("noepi")
		MasterTable.append(miEDFReader.readSignal(0,500,2500))
		miEDFReader.close


#ahora los epi
#print ("ficheros que va a tratar de epilepticos")
Epi=lsAb("/test_epis") #carpeta con archivos edf de epilepticos para testar el modelo
#ListaEdEpifReader =[]
for file in Epi:
	if (file!="/test_epis/.DS_Store"):
#		print(file)
		contador_test+=1
		miEDFReader = pyedflib.EdfReader(file)
		Etiqueta.append("epi")
		MasterTable.append(miEDFReader.readSignal(0,500,2500))
		miEDFReader.close   
#ahora viene el trabajo de IA

In [137]:
print(contador_train, contador_test)

176 131


In [138]:
MasterTablePanda = pd.DataFrame(MasterTable)
df = MasterTablePanda

In [139]:
normalized_df=(df-df.min())/(df.max()-df.min())
df=normalized_df


In [39]:
#Tratar datos

In [40]:
#PCA para disminuir número de columnas
#Si hacemos esto requiere separar previamente la columa final de EPI/NO EPI,
#para asegurarnos de que no la cambie



In [140]:
X=df
y=Etiqueta

pca= PCA(n_components=0.99999,random_state=RANDOM_STATE).fit(X)
X_pca=pca.transform(X)
X_pca.shape

(307, 268)

In [199]:
print(pd.DataFrame(X_pca))

          0         1         2         3         4         5         6    \
0   -0.656332 -0.087269  0.044535 -0.069131 -0.051735 -0.049591 -0.029009   
1   -0.624887  0.494315 -0.287538 -0.015886 -0.048259 -0.546252  0.476791   
2   -0.288607 -0.269890  0.285465 -0.056370 -0.087980 -0.047560 -0.164956   
3   -0.266502 -0.105824 -0.025454 -0.058957 -0.086618 -0.010630 -0.051306   
4    0.044643 -0.098184 -0.052907 -0.057531 -0.054141  0.009260 -0.041913   
..        ...       ...       ...       ...       ...       ...       ...   
302 -0.095197 -0.086243 -0.070573 -0.036673 -0.039974  0.020369 -0.027827   
303 -0.218737 -0.070856 -0.085148 -0.209112 -0.088360  0.002729  0.034585   
304  0.873790  0.041371 -0.131546 -0.057427 -0.017168  0.042299  0.011670   
305 -0.036377 -0.132175 -0.095836 -0.050262 -0.064487  0.029353 -0.002239   
306  0.497493 -0.150010  0.231288 -0.043499 -0.020234 -0.115177 -0.020070   

          7         8         9    ...           258       259       260  \

In [141]:
#Random Forest splitting the data

def split_vals(a,n): return a[:n].copy(), a[n:].copy()

n_valid = contador_test
n_trn = len(df)-n_valid
raw_train, raw_valid = split_vals(df, n_trn)
X_train, X_valid = split_vals(X_pca, n_trn)
#X_train, X_valid = split_vals(X, n_trn)
y_train, y_valid = split_vals(y, n_trn)

X_train.shape, X_valid.shape

((176, 268), (131, 268))

In [44]:
#Montar Random Forest

In [206]:
m = RandomForestClassifier(n_estimators=5, min_samples_leaf=2, max_features=8, n_jobs=-1)  #builds the model with number of trees, min number of samples per leaf and maximum number of features taken into account
m.fit(X_train, y_train) #trains the RF with our data


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features=8, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=2, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5, n_jobs=-1,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [207]:
trainscore = m.score(X_train, y_train)
predscore = m.score(X_valid, y_valid)
print(trainscore,predscore) #-> prints the accuracy of the model in training and testing


0.9602272727272727 0.5419847328244275


In [195]:
#pd.crosstab(y_valid, m.predict(X_valid)) #-> confusion matrix to see how often it is right (indicates visually how often the prediction and actual value coincide)

confusion_matrix(y_valid, m.predict(X_valid))


array([[42, 22],
       [34, 33]])